<a href="https://colab.research.google.com/github/ianomunga/backTrack/blob/main/twythonStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from twython import Twython, TwythonError, TwythonStreamer
import pandas as pd
from datetime import datetime
import csv
import os

In [ ]:
#My sensitive Twitter API Data is redacted

twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
twitter.search(q='covid', result_type='popular')
#to test run the OAuth and confirm whether authentication was successful

In [ ]:
from twython import TwythonStreamer
def process_tweet(tweet): 
    
    filtered_data = {}
    dict_test = []
    
    initial_format = '%a %b %d %H:%M:%S %z %Y'
    date_formats = '%d-%m-%Y'
    time_formats = '%H:%M:%S'
    
    filtered_data['id_post'] = tweet['id']
    filtered_data['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']] 
    filtered_data['date'] = datetime.strptime(tweet['created_at'], initial_format).strftime(date_formats)
    filtered_data['time'] = datetime.strptime(tweet['created_at'], initial_format).strftime(time_formats)
    filtered_data['geo'] = tweet['geo']
    filtered_data['text'] = tweet['text'] 
    filtered_data['user'] = tweet['user']['screen_name'] 
    filtered_data['user_loc'] = tweet['user']['location'] 
    filtered_data['user_id'] = tweet['in_reply_to_user_id']
    filtered_data['source_device'] = tweet['source']
                                   
    dict_test.append(filtered_data)
    
    print(dict_test)

    return dict_test

class MyStreamer(TwythonStreamer):
    
    def on_success(self, data):
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data) 
            
    def on_error(self, status_code, data):
        print(status_code)
        
    def save_to_csv(self, tweet, encoding = 'utf-8'):
        file_name = 'Twitter_{date}.csv'.format(date = str(datetime.now().strftime('%d-%m-%Y')))
        try:
            with open(file_name, 'a', newline='', encoding=encoding) as file:
                size_path_file =  os.path.getsize(file_name)
                print(size_path_file)
                if size_path_file == 0:
                    writer = csv.DictWriter(file, fieldnames=tweet[0].keys())
                    writer.writeheader()
                    for data in tweet:
                        writer.writerow(data)
                else:
                    writer = csv.DictWriter(file, fieldnames=tweet[0].keys())
                    for data in tweet:
                        writer.writerow(data)
                        
        except IOError:
            print("I/O error")


In [ ]:
"""
-----For when I'll need to pop some on the fly-----
results = twitter.cursor(twitter.search, q='bush')
for result in results: 
  print(result)
"""

In [ ]:
if __name__ == '__main__':
    stream = MyStreamer(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    stream.statuses.filter(track=['Covid', 'Covid-19'])